In [14]:
import collections

In [15]:
def typeA_dict(string):
    '''
    Input: A string of words
    Output: A dictionary with keys as verb roots and values as Type-A stem extenders
    '''
    typeA_words = {} #dictionary with words as keys and extenders as values
    for i in string.split():
        if i[-1] == 'ു' and i[-8:-5] != 'ക്ക' and 'ുന്നു' not in i and 'ുകയാണു' not in i: #getting the Type-A past forms that end in -u, filtering out Type-B endings
            stem = i[:-1] #stem is the word minus the past affix
            extender = stem[-1] #extender is the last character in the stem 
            if extender == stem[-3] or stem[-3] == 'ന': #for geminates like ɲɲ or nn
                extender =  stem[-3:]
            #root is stem minus extender; the 1 arg removes the last occurence of the extender characater from the stem for words like 'തൊഴുതു' where 'ത' occurs twice 
            root = ''.join(stem.rsplit(extender, 1))
            typeA_words[root] = extender
        elif i.endswith('ിട്ടുണ്ട്'): #getting the Type-A perfective forms that end in -iʈʈunɖe
            stem = i[:-9]
            if stem[-1] == 'യ' or stem[-1] == 'വ': #filtering out epenthetic forms like /y/ or /v/ before /ittunde/
                stem = stem[:-1]
            extender = stem[-1]
            if extender == stem[-3] or stem[-3] == 'ന': #for geminates
                extender =  stem[-3:]
            root = ''.join(stem.rsplit(extender, 1))
            typeA_words[root] = extender
    return typeA_words
#Note: this dictionary does not allow duplicates, so if both 'ചെയ്തു' and 'ചെയ്തിട്ടുണ്ട്' exist, only one of them will 
#be added to the dictionary; this serves our purpose.

In [16]:
def typeB_dict(string):
    '''
    Input: A string of words
    Output: A dictionary with keys as verb roots and values as Type-B stem extenders
    '''
    typeB_ending = ['ുന്നു', 'ും', ' ുക', 'ുകയാണു'] #list containing type-b forms
    temp_list = [] #temporary list for raw type-B words
    typeB_words ={} #dictionary with words as keys and extenders as values
    for i in string.split():
        temp_list.append([i for ending in typeB_ending if ending in i]) 

    temp_list = [x for x in temp_list if x != []] #to get rid of empty lists
    temp_list = [item for sublist in temp_list for item in sublist] #to get rid of nested lists
 
    for word in temp_list:
        for ending in typeB_ending:
            if word.endswith(ending):
                stem = word.strip(ending) 
                if stem[-1] == 'യ' or stem[-1] == 'വ': #filtering out epenthetic forms like /j/ before /ittunde/
                    stem = stem[:-1]
        if stem.endswith('ക്ക'): #two kinds of type-B extenders, either kk or null
            extender = 'ക്ക' 
        else:
            extender = '0'
        root = stem.rstrip(extender) 
        typeB_words[root] = extender
    return typeB_words

In [17]:
def sort_dict(dict):
    '''
    Input: Takes a unsorted dictionary
    Output: Returns a sorted dictionary
    '''
    
    temp_sorted = sorted(dict.items(), key=lambda kv: kv[1])
    sorted_dict = collections.OrderedDict(temp_sorted)
    return sorted_dict

In [18]:
def match_items(typeA_words, typeB_words):
    '''
    Input: Takes two dictionaries
    Output: Returns two dictionaries such that only the keys that are shared between the two are kept in both 
    '''
    matchtypeA={}
    matchtypeB={}
    for key in typeA_words.keys() & typeB_words.keys(): #if root in both dictionaries
        matchtypeA[key] = typeA_words[key]
        matchtypeB[key] = typeB_words[key]
    return(matchtypeA, matchtypeB)


In [19]:
def score(typeA, typeB):
    '''
    Input: Takes two dictionaries with keys as stems and values as extenders
    Output: Returns score equal to no. of groups of stems that share type-A extender and same type-B extender 
    divided by total number of groups
    '''
    grouped_ext = {} #dictionary with key as type-A extender and list of values as stems with that extender
    for key, value in sorted(typeA.items()):
        grouped_ext.setdefault(value, []).append(key)
    groups = dict(grouped_ext)

    extB_same = [] #list of type-B extenders for each group. For ex: for a group [cheyy, peyy], extB_same = [0,0]
    match = 0
    for val_group in groups.values():
        extB_same = []
        for val in val_group:
            extB = typeB[val]
            extB_same.append(extB)
        if len(extB_same) > 1: #only if group of type-B extenders has more than one element
            if any(ext == extB_same[0] for ext in extB_same): #if all elements in group of type-B extenders are equal
                match += 1
    return match

In [20]:
with open ("D:\\IndicNLP\\data_500\\verbs_final.txt", "r", encoding='utf-8') as in_fp:
    string = in_fp.read().replace('\n', ' ')
typeA_words = typeA_dict(string)
typeB_words = typeB_dict(string)
matchtypeA, matchtypeB = match_items(typeA_words, typeB_words)
typeA, typeB = sort_dict(matchtypeA), sort_dict(matchtypeB)
print("List of shared Type-A roots and extenders", matchtypeA)
print("List of shared Type-B roots and extenders", matchtypeB)
print("Score = " , score(typeA, typeB))

List of shared Type-A roots and extenders {'ചെയ്': 'ത', 'അനുവദി': 'ച്ച', 'സാധി': 'ച്ച', 'വിളി': 'ച്ച', 'കുറ': 'ഞ്ഞ', 'ചാ': 'ഞ്ഞ', 'ചോദി': 'ച്ച', 'കുതി': 'ച്ച', 'സ്നേഹി': 'ച്ച', 'കാണി': 'ച്ച', 'വിശദീകരി': 'ച്ച', 'ശ്രദ്ധി': 'ച്ച', 'ആരംഭി': 'ച്ച', 'കഴി': 'ഞ്ഞ', 'പറ': 'ഞ്ഞ'}
List of shared Type-B roots and extenders {'ചെയ്': '0', 'അനുവദി': 'ക്ക', 'സാധി': 'ക്ക', 'വിളി': 'ക്ക', 'കുറ': '0', 'ചാ': '0', 'ചോദി': 'ക്ക', 'കുതി': 'ക്ക', 'സ്നേഹി': 'ക്ക', 'കാണി': 'ക്ക', 'വിശദീകരി': 'ക്ക', 'ശ്രദ്ധി': 'ക്ക', 'ആരംഭി': 'ക്ക', 'കഴി': '0', 'പറ': '0'}
Score =  2
